In [1]:
import numpy as np
from scipy.special import softmax

In [2]:
def getTopKMask(x, k):
    N = x.shape[0]
    xSorted = sorted(list(enumerate(x)), key=lambda elem : abs(elem[1]), reverse=True)
    mask = np.zeros(N)
    for i in range(k):
        mask[xSorted[i][0]] = 1
    return mask

In [3]:
def topK(gradient0, k):
    gradient = gradient0.copy()
    gradient *= getTopKMask(gradient, k)
    return gradient

In [12]:
def smartCompression(gradient0, probability, k, alpha=0.5):
    gradient = gradient0.copy()
    N = gradient.shape[0]
    mask = getTopKMask(probability, k)
    gradient *= mask
    probability *= (np.ones(N) - mask * alpha)
    probability = softmax(probability)
    return gradient, probability

In [5]:
a = np.arange(1, 11, dtype=np.double)
topK(a, 5)

array([ 0.,  0.,  0.,  0.,  0.,  6.,  7.,  8.,  9., 10.])

In [6]:
smartCompression(a, a, 5)

(array([ 0.,  0.,  0.,  0.,  0.,  6.,  7.,  8.,  9., 10.]),
 array([1. , 2. , 3. , 4. , 5. , 3. , 3.5, 4. , 4.5, 5. ]))

In [7]:
sigma = lambda x : 1 / (1 + np.exp(-x))

def gradf(theta, X, y):
    N, dim = X.shape
    res = np.zeros(dim)
    for i in range(N):
        res += (y[i] - sigma(theta @ X[i].T)) * X[i]
    return res

def calculateLoss(theta, X, y):
    N, dim = X.shape
    res = 0
    for i in range(N):
        res += y[i] * np.log(sigma(theta @ X[i].T)) + (1 - y[i]) * np.log(1 - sigma(theta @ X[i].T))
    return res

In [19]:
def GradientDescent(gradf, theta0, X, y, max_iter=1000000, tol=1e-2, compression=None):
    theta = theta0.copy()
    iteration = 0
    gradients = []
    conv_array = []
    loss = []

    while True:
        alpha = 0.04
        conv_array.append(theta)
        loss.append(calculateLoss(theta, X, y))
        gradient0 = gradf(theta, X, y)
        gradient = gradient0.copy()
        gradients.append(np.linalg.norm(gradient0))
        if compression:
            gradient = compression(gradient)
        theta = theta + alpha * gradient

        iteration += 1
        if np.linalg.norm(gradient0) < tol:
            break
        if iteration >= max_iter:
            break
    res = {
        "num_iter": iteration,
        "loss": loss,
        "gradients": gradients,
        "tol": np.linalg.norm(gradient),
        "conv_array": conv_array,
    }
    return res

In [17]:
def SmartGradientDescent(gradf, theta0, X, y, k, max_iter=1000000, tol=1e-2):
    theta = theta0.copy()
    iteration = 0
    gradients = []
    conv_array = []
    loss = []
    probability = None

    while True:
        alpha = 0.04
        conv_array.append(theta)
        loss.append(calculateLoss(theta, X, y))
        gradient0 = gradf(theta, X, y)
        gradient = gradient0.copy()
        gradients.append(np.linalg.norm(gradient0))
        
        if iteration == 0:
            probability = gradient.copy()
        gradient, probability = smartCompression(gradient, probability, k)
        theta = theta + alpha * gradient

        iteration += 1
        if np.linalg.norm(gradient0) < tol:
            break
        if iteration >= max_iter:
            break
    res = {
        "num_iter": iteration,
        "loss": loss,
        "gradients": gradients,
        "tol": np.linalg.norm(gradient),
        "conv_array": conv_array,
    }
    return res